In [1]:
import os
import django
from datetime import timedelta, datetime

os.environ['DJANGO_SETTINGS_MODULE'] = 'cashup_backend.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

django.setup()

In [2]:
from data.models import HourData, MinuteData
from trade.models import TradeResult, TradeSetting
from django.db.models import Q

In [3]:
minute_data = MinuteData.objects.filter(
            datetime__range=[MinuteData.objects.last().datetime - timedelta(hours=6),
                             MinuteData.objects.last().datetime - timedelta(minutes=5)])

2021-08-23 12:45:00


In [3]:
def version_3_is_buy(flag, data):
    # set Default Value
    up_count = 0
    down_count = 0
    last_fu_time = ""
    last_fu_flag = False
    last_fd_time = ""
    last_fd_flag = False

    for element in data:
        if last_fu_time != "":
            if element.datetime - last_fu_time >= timedelta(minutes=60):
                last_fu_flag = True
                if element.up_down == "U":
                    up_count += 1

        if last_fd_time != "":
            if element.datetime - last_fd_time >= timedelta(minutes=60):
                last_fd_flag = True
                if element.up_down == "D":
                    down_count += 1

        if element.signal == "fU(U)":
            last_fu_flag = False
            last_fu_time = element.datetime
            up_count = 0

        if element.signal == "fD(D)":
            last_fd_flag = False
            last_fd_time = element.datetime
            down_count = 0

    if flag:
        if last_fu_flag and up_count > 2:
            return True

    else:
        if last_fd_flag and down_count > 2:
            return True
    return False

In [4]:
def version_2_is_buy(flag, data):
    last_signal = ""
    last_up_down = ""
    for element in data:
        last_signal = element.signal
        last_up_down = element.up_down

    if flag:
        if last_signal == 'fD(D)' and last_up_down == "U":
            return True
    else:
        if last_signal == 'fU(U)' and last_up_down == "D":
            return True
    return False

In [5]:
minute_data = MinuteData.objects.filter(datetime__range=[MinuteData.objects.last().datetime - timedelta(hours=6), MinuteData.objects.last().datetime])
now_price = minute_data.last().close_price
long_3 = version_3_is_buy(True, minute_data)
short_3 = version_3_is_buy(False, minute_data)
long_2 = version_2_is_buy(True, minute_data)
short_2 = version_2_is_buy(False, minute_data)
long_1 = False

OperationalError: (1054, "Unknown column 'data_minutedata.flow' in 'field list'")

In [ ]:
## 유저 확인 로직
test_trade_users = []
real_trade_users = []
user_query = TradeSetting.objects.all()
for element in user_query:
    if element.test_trade:
        test_trade_users.append({
            'user': element.user,
            'version': element.version,
            'buy_rate_option': element.buy_rate_option,
            'sell_rate_option': element.sell_rate_option
        })
    if element.real_trade:
        real_trade_users.append({
            'user': element.user,
            'version': element.version,
            'buy_rate_option': element.buy_rate_option,
            'sell_rate_option': element.sell_rate_option
        })

In [ ]:
account = test_trade_users[0]
## 구매 확인 및 판매 확인 로직
buy_check_query = TradeResult.objects.filter(Q(user=account['user']), ~Q(buy_order_time=None), Q(buy_time=None)).last()
if buy_check_query:
    if now_price < buy_check_query.buy_price:
        buy_check_query.buy_time = datetime.now()
        buy_check_query.save()

    elif datetime.now() - buy_check_query.buy_order_time > timedelta(hours=1):

        buy_check_query.delete()

sell_check_query = TradeResult.objects.filter(Q(user=account['user']), ~Q(sell_order_time=None), Q(sell_time=None))
if sell_check_query:
    if sell_check_query.last().sell_price < now_price:
        for element in sell_check_query:
            element.sell_time = datetime.now()
            element.calculate_earning_rate(now_price=element.sell_price)
            element.save()

In [ ]:
## 구매 로직
for account in test_trade_users:
    result_query = TradeResult.objects.filter(user=account['user'], sell_order_time=None)
    buy_flag = False
    if account['version'] == 1:
        if long_1:
            buy_flag = True
    elif account['version'] == 2:
        if long_2:
            buy_flag = True
    elif account['version'] == 3:
        if long_3:
            buy_flag = True

    if buy_flag:
        # 최초 구매 시
        if len(result_query) == 0:
            TradeResult.objects.create(
                user=account['user'],
                buy_order_time=datetime.now(),
                amount=100,
                buy_price=now_price * (1 - (account['buy_rate_option'] / 10000))
            )
        elif len(result_query) > 0:
            prev_trade = result_query.last()
            prev_amount, prev_price = prev_trade.amount, prev_trade.buy_price

            if now_price * (1 - (account['buy_rate_option'] / 10000)) < prev_price:
                TradeResult.objects.create(
                    user=account['user'],
                    buy_order_time=datetime.now(),
                    amount=prev_amount,
                    buy_price=now_price * (1 - (account['buy_rate_option'] / 10000))
                )

In [ ]:
## 판매 호가 로직
sell_query = TradeResult.objects.filter(Q(user=account['user']), ~Q(buy_time=None), Q(sell_order_time=None))
earning_rate_sum = 0
if sell_query:
    for element in sell_query:
        element.calculate_earning_rate(now_price=now_price)
        earning_rate_sum += element.earning_rate
        element.save()

if earning_rate_sum > 0:
    for element in sell_query:
        element.sell_order_time = datetime.now()
        element.sell_price = now_price * (1 + (account['sell_rate_option'] / 10000))
        element.save()
